In [ ]:
import os
from PIL import Image, ImageOps
import numpy as np
from paddleocr import PaddleOCR
import re
import string
import json
from langchain.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# 1. 이미지 폴더 경로 설정 및 JSON 저장 폴더 생성
img_folder_path = r'C:/Project/Python_basic/it/기술IT인터넷테크놀로지'  # 이미지가 저장된 폴더 경로
json_folder_path = r'json_results_it/'  # JSON 파일이 저장될 폴더 경로

# JSON 폴더가 존재하지 않으면 생성
if not os.path.exists(json_folder_path):
    os.makedirs(json_folder_path)

# 2. PaddleOCR 초기화
ocr = PaddleOCR(lang='korean', use_gpu=True)

# 3. 이미지 폴더 내의 모든 이미지 파일 처리
for img_filename in os.listdir(img_folder_path):
    if img_filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        img_path = os.path.join(img_folder_path, img_filename)

        # 4. 이미지 로드 (Pillow 사용)
        try:
            with Image.open(img_path) as img:
                image = np.array(img)
        except Exception as e:
            print(f"Warning: Failed to load image {img_filename}. Exception: {e}. Skipping this file.")
            continue

        # 5. 이미지 밝기 계산 (평균 밝기)
        gray_image = ImageOps.grayscale(img)
        mean_brightness = np.mean(np.array(gray_image))

        # 6. 조건부 색 반전: 어두운 배경(평균 밝기 낮음)인 경우에만 색 반전
        if mean_brightness < 127: 
            # 색 반전 수행
            print(f"Image {img_filename} has a dark background. Inverting colors.")
            inverted_image = ImageOps.invert(img)
        else:
            # 색 반전이 필요 없으면 원본 이미지 사용
            print(f"Image {img_filename} has a light background. No inversion needed.")
            inverted_image = img

        # 7. 반전된 이미지 또는 원본 이미지를 임시 경로에 저장
        inverted_image_path = os.path.join(json_folder_path, f'inverted_{img_filename}')
        inverted_image.save(inverted_image_path)

        # 8. 이미지에서 텍스트 추출 (OCR)
        result = ocr.ocr(inverted_image_path, cls=False)
        ocr_result = result[0]

        if not result or not result[0]:
            print(f"Warning: OCR failed for {img_filename}.")
            continue

        ocr_result = result[0]

        # 9. 텍스트와 좌표를 추출하고 y 좌표로 정렬
        y_threshold = 20
        sorted_results = sorted([(item[1][0], item[0][0][0], item[0][0][1]) for itemb in ocr_result], key=lambda x: x[2])

        # 10. y 좌표를 기준으로 그룹화
        groups = []
        current_group = [sorted_results[0]]

        for item in sorted_results[1:]:
            if abs(item[2] - current_group[-1][2]) <= y_threshold:
                current_group.append(item)
            else:
                groups.append(current_group)
                current_group = [item]

        groups.append(current_group)

        # 11. 특정 기호를 제거하는 함수 정의
        def clean_text(text):
            pattern = f"[{re.escape(string.punctuation)}]"
            cleaned_text = re.sub(pattern, '', text)
            cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
            return cleaned_text

        # 12. 각 그룹 내에서 x 좌표로 정렬하고 텍스트 연결 (기호 제거 적용)
        result = []
        for group in groups:
            sorted_group = sorted(group, key=lambda x: x[1])
            line_data = [{'text': clean_text(item[0]), 'coordinates': item[1]} for item in sorted_group]
            result.append(line_data)

        # 13. 결과를 JSON 파일로 저장
        output_data = {
            "lines": result
        }

        output_json_path = os.path.join(json_folder_path, f'{os.path.splitext(img_filename)[0]}_ocr_result.json')
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, ensure_ascii=False, indent=4)

        # 14. JSON 파일 불러오기 및 텍스트 추출
        texts = []
        for line in output_data['lines']:
            line_text = ' '.join([item['text'] for item in line])
            texts.append(line_text)

        full_text = ' '.join(texts)

        # 15. 요약을 위한 프롬프트 설정
        prompt_template = """
        당신은 도움이 되는 이미지 텍스트분석전문가입니다. 
        다음 텍스트를 요약하되, 주요 포인트에 중점을 두고 아래와 같은 형식으로 요약하세요:
        요약에 집중하고 주요 포인트를 간단하게 나타내주세요
        용량 과 단위도 나타내세요

        1. 요약:
        2. 주요 포인트:
            - 포인트 1
            - 포인트 2
            - 포인트 3

        요약할 내용이 없다면, 요약을 생략해도 됩니다.
        요약이 불가능하다면 이유를 말해주세요
        다음은 요약할 텍스트입니다:


        {input_text}
        """

        prompt = ChatPromptTemplate.from_template(prompt_template)

        # 16. Ollama 모델 로드 및 LLMChain 설정
        model = ChatOllama(model='llama3.1:latest')
        llm_chain = LLMChain(llm=model, prompt=prompt)

        # 17. 텍스트 요약 수행
        summary = llm_chain.run(input_text=full_text)

        # 18. 요약된 내용 출력
        print(f"Original Text from {img_filename}:\n", full_text)
        print(f"\nSummarized Text from {img_filename}:\n", summary)
